In [1]:
!nvidia-smi

Sun Nov 21 20:19:27 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.67       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 3090    Off  | 00000000:65:00.0 Off |                  N/A |
| 73%   54C    P8    52W / 370W |   1581MiB / 24265MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 3090    Off  | 00000000:B3:00.0 Off |                  N/A |
| 57%   

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import skimage.transform as st
import os
import gc
import warnings
from sklearn.metrics import classification_report, average_precision_score
from Parse_TFrecords import *
from define_model import *
from load_data import *
from utilities import *
 
print(tf.__version__)

warnings.filterwarnings("ignore")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

gpus = tf.config.list_physical_devices(device_type='GPU')
tf.config.set_visible_devices(devices=gpus[0], device_type='GPU')

2.5.0


# Pretrain model

In [3]:
files = '../Data/Chexpert_cxr.tfrecords'

BATCH_SIZE = 32

train_dataset = (tf.data.TFRecordDataset(
    files, compression_type=None, buffer_size=BATCH_SIZE*10, num_parallel_reads=32)
.map(parse_TFrecord_pretrain)
.batch(BATCH_SIZE, drop_remainder=True)
.shuffle(BATCH_SIZE*10, reshuffle_each_iteration=True))

In [6]:
checkpoint_filepath = 'checkpoints/Pretrain/checkpoint_pretrain_InceptionV3'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only=True,
    save_weights_only=True,
    monitor='auc',
    mode='auto')

callback = [tf.keras.callbacks.LearningRateScheduler(scheduler),
            tf.keras.callbacks.EarlyStopping(mode='auto', patience=3, monitor='auc'),
            model_checkpoint_callback]

model = define_model_pretrain(archi='IV3')

model.summary()

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                 optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3))

model.fit(train_dataset, epochs=10, shuffle=True, callbacks=callback)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d_94 (Conv2D)              (None, 127, 127, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_94 (BatchNo (None, 127, 127, 32) 96          conv2d_94[0][0]                  
__________________________________________________________________________________________________
activation_94 (Activation)      (None, 127, 127, 32) 0           batch_normalization_94[0][0]     
______________________________________________________________________________________________

Epoch 1/10
4403/4403 [==============================] - 377s 83ms/step - loss: 0.5031
Epoch 2/10
4403/4403 [==============================] - 368s 83ms/step - loss: 0.4880
Epoch 3/10
4403/4403 [==============================] - 368s 83ms/step - loss: 0.4719
Epoch 4/10
4403/4403 [==============================] - 369s 84ms/step - loss: 0.4632
Epoch 5/10
4403/4403 [==============================] - 369s 84ms/step - loss: 0.4543
Epoch 6/10
4403/4403 [==============================] - 369s 84ms/step - loss: 0.4479
Epoch 7/10
4403/4403 [==============================] - 368s 83ms/step - loss: 0.4367
Epoch 8/10
4403/4403 [==============================] - 368s 83ms/step - loss: 0.4273
Epoch 9/10
4403/4403 [==============================] - 368s 83ms/step - loss: 0.4138
Epoch 10/10
4403/4403 [==============================] - 367s 83ms/step - loss: 0.3981


In [ ]:
checkpoint_filepath = 'checkpoints/Pretrain/checkpoint_pretrain_Dnet121'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only=True,
    save_weights_only=True,
    monitor='loss',
    mode='auto')

callback = [tf.keras.callbacks.LearningRateScheduler(scheduler),
            tf.keras.callbacks.EarlyStopping(mode='auto', patience=3, monitor='loss'),
            model_checkpoint_callback]

metric = tf.keras.metrics.PrecisionAtRecall(recall=0.5, name='precision_at_recall')

model = define_model_pretrain(archi='Dnet121')

model.summary()

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                 optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), metrics=metric)

model.fit(train_dataset, epochs=10, shuffle=True, callbacks=callback)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 128, 128, 64) 9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 128, 128, 64) 256         conv1/conv[0][0]                 
______________________________________________________________________________________________

Epoch 1/10
   1779/Unknown - 241s 128ms/step - loss: 0.5701 - precision_at_recall: 0.6148

# Transfer learning

In [3]:
pos = 27297
neg = 119133

total = pos+neg

weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

Weight for class 0: 0.61
Weight for class 1: 2.68


In [4]:
BATCH_SIZE = 32

record_file_train = 'copd_train_new.tfrecords'
train_dataset = (tf.data.TFRecordDataset(
    record_file_train, buffer_size=BATCH_SIZE*10, compression_type=None, num_parallel_reads=32)
.map(parse_TFrecord_train)
.shuffle(total)
.batch(BATCH_SIZE))

record_file_val = 'copd_val_new.tfrecords'
val_dataset = (tf.data.TFRecordDataset(
    record_file_val, buffer_size=BATCH_SIZE*10, compression_type=None, num_parallel_reads=32)
.map(parse_TFrecord_train)
.shuffle(36601)
.batch(BATCH_SIZE))

record_file_test = 'copd_test_new.tfrecords'
test_dataset = (tf.data.TFRecordDataset(
    record_file_test, buffer_size=BATCH_SIZE*10, compression_type=None, num_parallel_reads=32)
.map(parse_TFrecord_test)
.batch(BATCH_SIZE))

y_train = get_data_label('train')
y_val = get_data_label('val')
y_test = get_data_label('test')

## Different loss functions, metrics

In [5]:
BATCH_SIZE = 32
archi='IV3'

checkpoint_filepath = 'checkpoints/AUC/checkpoint_BCE_IV3'
monitor_ = 'val_auc'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only=True,
    save_weights_only=True,
    monitor=monitor_,
    mode='max')
    
callback = [tf.keras.callbacks.LearningRateScheduler(scheduler),
            tf.keras.callbacks.EarlyStopping(mode='max', patience=3, monitor=monitor_),
            model_checkpoint_callback]

model = load_model_from_pretrain(archi)

loss_func = tf.keras.losses.BinaryCrossentropy(from_logits=True)
# loss_func = binary_focal_loss
# loss_func = tf.keras.losses.CategoricalHinge()
# loss_func = tf.keras.losses.KLDivergence()

# metric = tf.keras.metrics.PrecisionAtRecall(recall=0.5, name='precision_at_recall')
# metric = tf.keras.metrics.Recall(name='recall')
# metric = tf.keras.metrics.Precision(name='precision')
metric = tf.keras.metrics.AUC()

model.compile(loss=loss_func,
                 optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), metrics=metric)

model.fit(train_dataset, epochs=20, shuffle=True, validation_data=val_dataset, callbacks=callback, class_weight=class_weight)

model.load_weights(checkpoint_filepath)

gc.collect()

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Epoch 1/20
4576/4576 [==============================] - 491s 103ms/step - loss: 0.6614 - auc: 0.6670 - val_loss: 0.6342 - val_auc: 0.6531
Epoch 2/20
4576/4576 [==============================] - 478s 103ms/step - loss: 0.6302 - auc: 0.7102 - val_loss: 0.5932 - val_auc: 0.7043
Epoch 3/20
4576/4576 [==============================] - 478s 103ms/step - loss: 0.6140 - auc: 0.7315 - val_loss: 0.7531 - val_auc: 0.6812
Epoch 4/20
4576/4576 [==============================] - 479s 103ms/step - loss: 0.5982 - auc: 0.7490 - val_loss: 0.5148 - val_auc: 0.7190
Epoch 5/20
4576/4576 [==============================] - 479s 103ms/step - loss: 0.5788 - auc: 0.7677 - val_loss: 0.5660 - val_auc: 0.7248
Epoch 6/20
4576/4576 [==============================] - 479s 103ms/step - loss: 0.5602 - auc: 0.7855 - val_loss: 0.5592 - val_auc: 0.7160
Epoch 7/20
4576/4576 [============

173

In [13]:
archi='Dnet121'
checkpoint_filepath = 'checkpoints/AUC/checkpoint_BCE'

model = define_model(archi)

model.load_weights(checkpoint_filepath)

y_test = get_data_label()

y_preds = model.predict(test_dataset)

In [15]:
test_CI(y_preds, y_test)

thresh = get_thresh(y_test, y_preds, 'G-mean')

print(thresh)

print(classification_report(y_test, np.where(y_preds >= thresh, 1, 0)))
print(average_precision_score(y_test, y_preds, average=None))

gc.collect()

Sample auc mean: 0.73
Samole auc std: 0.00
Sample auc CI: 0.01
Confidence interval for AUC: [0.72 - 0.74]
0.43800864
              precision    recall  f1-score   support

           0       0.89      0.68      0.77     36808
           1       0.33      0.65      0.44      9026

    accuracy                           0.67     45834
   macro avg       0.61      0.67      0.61     45834
weighted avg       0.78      0.67      0.71     45834

0.4002152266928162


0